In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [2]:
import argparse
import torch
from torch import nn
import matplotlib.pyplot as plt
from stu import STU
import time
import random

Unable to import FlashFFTConv: No module named 'flashfftconv'. Falling back to PyTorch implementation.


In [4]:
def exponential_decay_init(size, lam=5.0):
    """
    Samples from an exponential distribution with rate lam, 
    then clips at 1, does (1 - clipped_value),
    and finally multiplies by ±1 with probability 1/2.
    """
    # 1) Sample uniform [0,1], convert to exponential
    u = torch.rand(size)
    x = -1.0 / lam * torch.log(1 - u)  # Exponential(λ = lam)

    # 2) Clip at 1
    x = torch.clamp(x, max=1.0)

    # 3) Subtract from 1 (to be nea.r 1 for small x)
    x = 1.0 - x  # Now we have distribution mostly near 1 for large lam

    # 4) Multiply by ±1 with prob 1/2
    sign = torch.sign(torch.randn(size))
    return x * sign

def compute_ar_x_preds(w: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
    d_out, d_in, k = w.shape
    b, l, d_in_x = x.shape
    assert d_in == d_in_x, (
        f"Dimension mismatch: w.shape={w.shape}, x.shape={x.shape}"
    )

    o = torch.einsum("oik,bli->bklo", w, x)

    for i in range(k):
        o[:, i] = torch.roll(o[:, i], shifts=i, dims=1)

    m = torch.triu(torch.ones(k, l, dtype=o.dtype, device=o.device))  # [k, l]
    m = m.unsqueeze(-1).repeat(1, 1, d_out)  # [k, l, d_out]

    ar_x_preds = torch.sum(o * m, dim=1)  # now shape is [b, l, d_out]

    return ar_x_preds

class LDS(nn.Module):
    def __init__(self, state_dim, input_dim, output_dim, kx=10):
        super(LDS, self).__init__()
        self.state_dim = state_dim
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.kx = kx
        self.h0 = nn.Parameter(torch.randn(state_dim))
        # init_A = torch.randn(state_dim)
        # self.A = nn.Parameter(init_A / torch.max(torch.abs(init_A)))

        # self.A = nn.Parameter((torch.rand(state_dim) * 0.2 + 0.8) * torch.sign(torch.randn(state_dim)))
        self.A = nn.Parameter(exponential_decay_init([state_dim], lam = 15))
        self.B = nn.Parameter(torch.randn(input_dim, state_dim) / input_dim)
        self.C = nn.Parameter(torch.randn(state_dim, output_dim) / state_dim)
        self.M = nn.Parameter(torch.randn(output_dim, input_dim, kx) / (output_dim))

    def forward(self, inputs):
        device = inputs.device
        bsz, seq_len, _ = inputs.shape
        h_t = self.h0.expand(bsz, self.state_dim).to(device)
        A = self.A.flatten()
        all_h_t = []
        for t in range(seq_len):
            u_t = inputs[:, t, :]
            h_t = A * h_t + (u_t @ self.B)
            all_h_t.append(h_t.unsqueeze(1))
        all_h_t = torch.cat(all_h_t, dim=1)
        lds_out = torch.matmul(all_h_t, self.C)

        ar = compute_ar_x_preds(self.M, inputs)
        return lds_out + ar

    def compute_loss(self, inputs, targets):
        mse_loss = nn.MSELoss()
        outputs = self(inputs)
        return mse_loss(outputs, targets)

In [8]:
layer_i = 0
state_dim = 768
batch_size = 10
epochs = 4000
seq_len = 1024
kx = 10
lr = 0.0001

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the layer i weights
stu_layer_full = torch.load(f"../stu_layers/stu_layer_{layer_i}_500m_param_full.pt", map_location=torch.device('cpu'))
stu_layer_full.to(device)

# Initialize LDS model
lds = LDS(state_dim, 768, 768, kx).to(device)
optimizer = torch.optim.Adam(lds.parameters(), lr=lr)

# Training
lds_loss_values = []

best_loss = float('inf')

/var/folders/42/5cczn_jx4454n6fvhcg5qbjw0000gn/T/ipykernel_83753/3556665184.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stu_layer_full = torch.load(f"../stu_layers/

In [9]:
for epoch in range(epochs):
    inputs = torch.randn(batch_size, seq_len, 768).to(device).to(torch.bfloat16)
    stu_outputs = stu_layer_full(inputs).to(device)

    optimizer.zero_grad()
    loss = lds.compute_loss(inputs.to(stu_outputs.dtype), stu_outputs)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(lds.parameters(), max_norm=1)
    lds_loss_values.append(loss.item())
    optimizer.step()

    with torch.no_grad():
        lds.A.data.clamp_(max=1, min=-1)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.1116056740283966
Epoch 10, Loss: 0.0946127325296402
Epoch 20, Loss: 0.08109938353300095
Epoch 30, Loss: 0.07006219029426575
